In [1]:
from pyspark.sql import functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1680514581471_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from py4j.protocol import Py4JJavaError
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import explode, col, udf, concat_ws, from_json, lit, array, expr, size
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import *
import json
import os
from pyspark.sql.types import BooleanType
import sys
import gc
print(sys.version)
# conf = SparkConf()
# sc = SparkContext(conf=conf)
# sc.setLogLevel("ERROR")

# spark = SparkSession.builder.appName("twitter_applications")\
#     .config("spark.sql.files.ignoreCorruptFiles", "true")\
#     .config('spark.executor.memory', '45G')\
#     .config('spark.driver.memory', '80G')\
#     .config('spark.driver.maxResultSize', '10G')\
#     .getOrCreate()

'''
    Step 3:
    Implement basic PageRank as described in 
    http://www.diva-portal.org/smash/get/diva2:1104337/FULLTEXT01.pdf Section 3.4.1
'''
input_bucket = 's3://cs5344-twitter-project/input/'
user_pair_rdd = spark.read \
     .csv(input_bucket+"user_pair_aws/*.csv").rdd
# user_pair_rdd = user_pair_rdd.map(lambda x: (x[0], (x[1], x[2])))
user_interact_rdd = spark.read \
     .csv(input_bucket+"user_interact_aws/*.csv").rdd
id_name = spark.read.option("header",True).csv(input_bucket+"id_name_dict_aws/*.csv")
id_name.show()
# if user A retweet B, the from_user = A, to_user = B
columns = ['from_user', 'to_user', 'retweet_ratio']
edge_rdd = user_pair_rdd.filter(lambda x: x[0] != x[1]).toDF(columns)  # remove all self pointed edges
# .join(user_interact_rdd).map(lambda x: (x[0], x[1][0][0], float(x[1][0][1]) / float(x[1][1]))) \

all_users = edge_rdd.select("from_user").union(edge_rdd.select("to_user")).distinct() \
    .withColumnRenamed("from_user", "users")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.7.16 (default, Dec 15 2022, 23:24:54) 
[GCC 7.3.1 20180712 (Red Hat 7.3.1-15)]
+-------------------+-------------------+
|        user_id_str|user_twitter_handle|
+-------------------+-------------------+
|1277653717467172864|          KonateEra|
| 889184194315866113|     ViniFerreirado|
|          296488613|              tenoq|
|1292411311612030977|    kamikaze1226617|
|          224175846|           Edunwa18|
|1181834654317531136|         MKtography|
|1241239744316690432|    SpeedDe86282699|
|          201088131|         Fiberblock|
|         4149402979|        luislara_97|
|         2418331615|            eptileR|
|1484471406985297925|           WabinabG|
|1278814483167633409|          jhendryx3|
| 934943098374823936|          shige_e_5|
|1475408590445989894|         TAzammalik|
|         2613144614|         AUTOMYSTIC|
|1482121649650548736|        alexhaujshu|
|1473274640805994497|    hoki2022bigwinx|
|         3173560541|      AshfordWomens|
|1233591105410625536|          jmfodd

In [ ]:
test_list = ['nintendo', 'pokemon', 'pokémon legends: arceus', 'pokémon', 'legend of arceus',
             'legend arceus', 'legends of arceus', 'pokemon legends: arceus', 'arceus']
# regex for filter
regex_pattern = "(?i)" + "|".join(test_list)
regex_pattern_hashtag = "(?i)" + "|".join([x.strip(' ') for x in test_list])

filter_df = raw_df.filter(raw_df.text.rlike(regex_pattern) | raw_df.text_hashtag.rlike(regex_pattern)).distinct()

In [5]:
auth_rdd = edge_rdd.select("*", lit(1).alias("auth"))
hub_rdd = edge_rdd.select("*", lit(1).alias("hub"))

for i in range(5):

    t = hub_rdd.withColumn("score", hub_rdd.retweet_ratio * hub_rdd.hub).groupby("to_user").agg(_sum("score").alias("a"))

    norm = t.rdd.map(lambda x: (1, x[1])).reduceByKey(lambda a, b: a + b).collect()[0][1]
    auth_rdd = auth_rdd.join(t, "to_user").select("from_user", "to_user", "retweet_ratio",
                                                  (col("a") / norm).alias("auth"))
    del t
    del norm
    gc.collect()

    # update hub
    t = auth_rdd.withColumn("score", auth_rdd.retweet_ratio * auth_rdd.auth).groupby("from_user").agg(
        _sum("score").alias("h"))

    norm = t.rdd.map(lambda x: (1, x[1])).reduceByKey(lambda a, b: a + b).collect()[0][1]
    hub_rdd = hub_rdd.join(t, "from_user").select("from_user", "to_user", "retweet_ratio",
                                                  (col("h") / norm).alias("hub"))
    del t
    del norm
    gc.collect()

auth_result = auth_rdd.join(id_name, id_name.user_id_str == auth_rdd.to_user).sort(col("auth").desc())\
    .select('to_user', 'auth', 'user_twitter_handle').distinct().limit(200)
# auth_result.show(50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
auth_result.sort(col("auth").desc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+-------------------+
|            to_user|                auth|user_twitter_handle|
+-------------------+--------------------+-------------------+
|1016822205785632768| 9.44569510037704E-8|      chibashi_game|
|1018767717531299840|2.278889490099215E-8|     butterflyxflew|
|1026363803741577216|7.196103411931095...|           gggoturi|
|1003949032606232576|5.535297361130571E-9|        MayroHuerta|
|1024025263586172928|3.273388628290853...|        LEGENDSLima|
|1010173616124846080|1.461702265236683...|         pixel_poem|
|1019274431410012160|6.619366891925705...|          koingyeal|
|1015469914390315009|5.737814942275034...|    LegendaryPrizes|
|         1013640066|4.966728687038877...|      augc_juridico|
|1033546544778117120|2.579141989770504...|        YuichiroKun|
|1005062109032480768|2.406579072688340...|            1UPconf|
|          100073363|1.303427937909756...|    MarcSeresCrypto|
|1024408162655293440|9.069428886899742...|      protect

In [7]:
# write to CSV
auth_result.sort(col("auth").desc()).limit(200).write.mode('overwrite').csv(input_bucket+"test_pokemon_hits")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
hub_result = hub_rdd.join(id_name, id_name.user_id_str == hub_rdd.from_user).sort(col("hub").desc())\
    .select('from_user', 'hub', 'user_twitter_handle').distinct().limit(200)
# write to CSV
hub_result.sort(col("hub").desc()).limit(200).write.mode('overwrite').csv(input_bucket+"test_pokemon_hits_hubs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…